In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sys
import os
import parsers as p
import time
%load_ext autoreload
%autoreload 2

In [72]:
timea=time.time()
df = pd.read_csv("../telemetry_exe/Raw_Data/data0101.csv", dtype = str, on_bad_lines = "skip")
init_sz = df.shape[0]
df = df[df["time"].str.len()==13]
df = df[df['msg.len'].str.len()==1]
df = df[~df['msg.id'].isnull()]
df = df[~(df['msg.id'].str.len()<2)]
df = df[~(df['msg.id'].str.len()>3)]

#df['time'] = pd.to_numeric(df['time'])
#print(np.any(df['time'][1:]-df['time'][:-1]<0))
#indices = np.where(df['time'][1:]-df['time'][:-1]<0)[0]+1
#df.iloc[indices]['time']=df['time'][indices]+1000
#df.iloc[265816-5:265816+5].head(10)

df['time'] = pd.to_datetime(df['time'], unit='ms')
df["msg.id"] = df["msg.id"].apply(int, base = 16)
df["msg.len"] = pd.to_numeric(df['msg.len'])
df = df[df['msg.len']*2 == df['data'].str.len()]
df["data"] = df["data"].apply(int, base = 16)
df["data"] = df['data'].to_numpy(dtype=np.uint64).byteswap()>>(8*(8-df["msg.len"])).to_numpy(dtype=np.uint16)
df = df.reset_index(drop=True)
pruned_sz = df.shape[0]
print(init_sz - pruned_sz)
print(time.time()-timea)

#df.head()

305
1.734907627105713


In [199]:
df = df.sample(frac=1).reset_index(drop=True)
df.sort_values(by=['time'],inplace=True)
df = df.reset_index(drop=True)

In [53]:
msg_id = df['msg.id'].to_numpy().astype(np.uint16)
msg_data = df['data'].to_numpy()

In [3]:
unique, freq = np.unique(msg_id, return_counts = True)
hexes = np.array([p.MESSAGE_DICT[i][1] for i in unique])
print(np.array((freq,hexes)).T)

[['36366' 'MC1_status']
 ['35320' 'MC2_status']
 ['35719' 'MC3_status']
 ['35866' 'MC4_status']
 ['34911' 'MC1_temps']
 ['34759' 'MC2_temps']
 ['34847' 'MC3_temps']
 ['34755' 'MC4_temps']
 ['9194' 'MC1_energy']
 ['8694' 'MC2_energy']
 ['8899' 'MC3_energy']
 ['8992' 'MC4_energy']
 ['9237' 'MC1_setpoints_command']
 ['8805' 'MC2_setpoints_command']
 ['9279' 'MC3_setpoints_command']
 ['8892' 'MC4_setpoints_command']
 ['3430' 'MCU_status']
 ['6489' 'BMS_onboard_temperatures']
 ['54322' 'BMS_voltages']
 ['26135' 'BMS_detailed_voltages']
 ['12952' 'BMS_temperatures']
 ['8067' 'BMS_detailed_temperatures']
 ['3355' 'BMS_status']
 ['341' 'Dashboard_status']
 ['3397' 'EM_status']
 ['3409' 'EM_measurement']]


In [124]:
timea=time.time()
df = pd.read_csv("../telemetry_exe/Raw_Data/data0005.csv", dtype = str, on_bad_lines = "skip")
init_sz = df.shape[0]
df = df[df["time"].str.len()==13]
df = df[df['msg.len'].str.len()==1]
df = df[~df['msg.id'].isnull()]
df = df[~(df['msg.id'].str.len()<2)]
df = df[~(df['msg.id'].str.len()>3)]
#df['time'] = pd.to_datetime(df['time'], unit='ms')
df['time'] = pd.to_numeric(df['time'])
df["msg.id"] = df["msg.id"].apply(int, base = 16)
df["msg.len"] = pd.to_numeric(df['msg.len'])
df = df[df['msg.len']*2 == df['data'].str.len()]
df["data"] = df["data"].apply(int, base = 16)
df["data"] = df['data'].to_numpy(dtype=np.uint64).byteswap()>>(8*(8-df["msg.len"])).to_numpy(dtype=np.uint16)
df = df.reset_index(drop=True)
pruned_sz = df.shape[0]
print("Removed messages: " + str(init_sz - pruned_sz))
timeb = time.time()
print("CSV parse time: " + str(timeb-timea))
msg_id = df['msg.id'].to_numpy().astype(np.uint16)
msg_data = df['data'].to_numpy()
msg_time = df['time'].to_numpy()
points = 0
messages = 0
for i in p.MESSAGE_DICT.values():
    try:
        a = i[0](msg_data, msg_id)
        points += a.shape[0]
        messages += 1
        #print(str(i[1]) + " " + str(a.max()))
    except:
        continue
#print(messages)
#print(points)
print("Message coverage: "+str(points/msg_id.size))
print("Message parse time: " + str(time.time()-timeb))
print("Total time: " + str(time.time()-timea))

#df.head()

Removed messages: 77
CSV parse time: 1.4650132656097412
Message coverage: 0.9773362833580971
Message parse time: 0.02888178825378418
Total time: 1.4938950538635254


In [36]:
print(p.parse_MC1_energy(msg_data, msg_id)[:,1].max())
print(p.parse_MC2_energy(msg_data, msg_id)[:,1].max())
print(p.parse_MC3_energy(msg_data, msg_id)[:,1].max())
print(p.parse_MC4_energy(msg_data, msg_id)[:,1].max())

12267.0
14501.0
18055.0
21382.0


In [165]:
def add_to_dir(root, directory, data):
    cwd = root
    for index, i in enumerate(directory[:-1]):
        if i not in cwd.keys():
            cwd[i] = {}
        cwd = cwd[i]
    cwd[directory[-1]] = data

def str_to_directory(s):
    s=s.replace("]","")
    l = re.split('[.]|[\/]|[\\]|[\[]', s)
    return [int(x) if x.isdigit() else x for x in l]

data = p.parse_BMS_detailed_temperatures(msg_data, msg_id, msg_time)
root = {}
for i, d in enumerate(data[0][3]):
    add_to_dir(root, str_to_directory(d), data[0][0][:, (0,i+1)])
print(json.dumps(root, sort_keys=True, indent=4, default=str))

{
    "BMS": {
        "detailed_temps": {
            "group": "[[1.546e+12 1.000e+00]\n [1.546e+12 0.000e+00]\n [1.546e+12 0.000e+00]\n ...\n [1.546e+12 1.000e+00]\n [1.546e+12 0.000e+00]\n [1.546e+12 1.000e+00]]",
            "id": "[[1.546e+12 1.000e+01]\n [1.546e+12 1.100e+01]\n [1.546e+12 0.000e+00]\n ...\n [1.546e+12 1.000e+01]\n [1.546e+12 1.100e+01]\n [1.546e+12 1.100e+01]]",
            "temps": {
                "0": "[[1.546e+12 2.365e+01]\n [1.546e+12 2.396e+01]\n [1.546e+12 2.373e+01]\n ...\n [1.546e+12 2.404e+01]\n [1.546e+12 2.429e+01]\n [1.546e+12 2.418e+01]]",
                "1": "[[1.546e+12 2.627e+01]\n [1.546e+12 2.391e+01]\n [1.546e+12 2.366e+01]\n ...\n [1.546e+12 2.406e+01]\n [1.546e+12 2.424e+01]\n [1.546e+12 2.406e+01]]",
                "2": "[[1.546e+12 0.000e+00]\n [1.546e+12 2.371e+01]\n [1.546e+12 2.360e+01]\n ...\n [1.546e+12 0.000e+00]\n [1.546e+12 2.405e+01]\n [1.546e+12 0.000e+00]]"
            }
        }
    }
}


In [159]:
data[0][0][:, (0,3)]

array([[1.546e+12, 2.365e+01],
       [1.546e+12, 2.396e+01],
       [1.546e+12, 2.373e+01],
       ...,
       [1.546e+12, 2.404e+01],
       [1.546e+12, 2.429e+01],
       [1.546e+12, 2.418e+01]])

In [ ]:
import plotly.graph_objects as go
a = p.parse_BMS_detailed_temperatures(msg_data, msg_id)
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,2], mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,3], mode='markers'))
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,4], mode='markers'))
fig.show()

In [80]:
%%timeit -n 1 -r 1
def add_to_dir(root, directory, data):
    cwd = root
    for index, i in enumerate(directory[:-1]):
        if i not in cwd.keys():
            cwd[i] = {}
        cwd = cwd[i]
    cwd[directory[-1]] = data

root = {}
directory = ["a","b","c",1,3]
data = np.array([1,2,3])
add_to_dir(root, directory, data)
directory = ["a","b","c",1,4]
data = "hello"
add_to_dir(root, directory, data)
root

17 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [71]:
import json 
print(json.dumps(root, sort_keys=True, indent=4, default=str))

{
    "a": {
        "b": {
            "c": {
                "1": {
                    "3": "[1 2 3]",
                    "4": "hello"
                }
            }
        }
    }
}


In [42]:
fig = go.Figure()
a = p.parse_MC1_energy(msg_data, msg_id)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
a = p.parse_MC2_energy(msg_data, msg_id)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
a = p.parse_MC3_energy(msg_data, msg_id)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
a = p.parse_MC4_energy(msg_data, msg_id)
fig.add_trace(go.Scatter(x=np.arange(a.shape[0])+1, y=a[:,1], mode='markers'))
fig.show()

In [45]:
import re

In [64]:
%%timeit
s = "a.b.c[2][1]"
s=s.replace("]","")
l = re.split('[.]|[\/]|[\\]|[\[]', s)
[int(x) if x.isdigit() else x for x in l]

1.7 µs ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


<function dir>

In [129]:
time.time()

1681329661.465289

In [ ]:
enumerate()